In [ ]:
def merge_sort(array, use_merge_improvement=False,
               use_insertion_sort=False, use_role_swap=False):
    """Merge sort (with optimizations)"""
    tmp_array = list(array)
    
    merge_sort_aux(array, tmp_array, 0, len(array) - 1, use_merge_improvement,
                   use_insertion_sort, use_role_swap)
    return array


In [ ]:
def merge_sort_aux(array_from, array_to, lo, hi,
                   use_merge_improvement=False,
                   use_insertion_sort=False,
                   use_role_swap=False):
    if hi <= lo:
        return

    mid = lo + (hi - lo) // 2
    merge_sort_aux(array_from, array_to, lo, mid, use_merge_improvement, use_insertion_sort, use_role_swap)
    merge_sort_aux(array_from, array_to, mid + 1, hi, use_merge_improvement, use_insertion_sort, use_role_swap)

    merge(array_from, array_to, lo, mid, hi, use_merge_improvement, use_role_swap)


In [ ]:
def merge(array_from, array_to, lo, mid, hi,
          use_merge_improvement=False, use_role_swap=False):
    
    i = lo
    j = mid + 1
    for k in range(lo, hi + 1):  # k = lo,...,hi
        if j > hi or (i <= mid and array_from[i] <= array_from[j]):
            array_to[k] = array_from[i]
            i += 1
        else:
            array_to[k] = array_from[j]
            j += 1
    for k in range(lo, hi + 1):  # k = lo,...,hi
        array_from[k] = array_to[k]

In [ ]:
# Sortiere array im Bereich lo bis hi mit Insertionsort
def insertion_sort(array, lo, hi):
    for i in range(lo, hi+1):
        val = array[i]
        j = i
        while j > lo and array[j - 1] > val:
            array[j] = array[j - 1]
            j -= 1
        array[j] = val

Ein einfaches Beispiel, das Sie verwenden können, die Korrektheit ihrer Implementierung mit verschiedenen aktivierten Verbesserungen zu testen (versuchen Sie in jedem Fall use_swap in Kombination mit den anderen beiden Verbesserungen).

In [ ]:
# hier ggf. ändern, um andere Konfigurationen zu testen
use_merge_improvement = True
use_insertion_sort = True
use_role_swap = True

test = [6, 2, 5, 8, 4, 5, 2, 4, 12, 3, 2, 5]
merge_sort(test, use_merge_improvement, use_insertion_sort, use_role_swap)

# Aufgabenteil d

Folgender Code enthält die Implementierung für die Experimente in Teil d. Sie können die Details ignorieren und einfach die Aufrufe weiter unten verwenden.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import random
import timeit


def sort_all_test_instances(test_instances, use_merge_improvement=False,
        use_insertion_sort=False, use_role_swap=False):
    """Hilfsfunktion für die Experimente.

    Sortiert alle test_instances mit merge_sort und den gegebenen Parametern
    für die verschiedenen Verbesserungen.
    """
    for array in test_instances:
        a = list(array) # copy to not change the instance for the next run
        merge_sort(a, use_merge_improvement, use_insertion_sort, use_role_swap)

        
def run_experiment(instance_creator):
    """Führe ein komplettes Experiment durch.

    Die Testarrays werden von instance_creator erstellt.
    """

    # Wir generieren zunächst Arrays verschiedener Grösse, und zwar von
    # min_size bis max_size mit jeweils Abstand step. Für jede Grösse erstellen
    # wir instances_per_size viele zufällige Arrays, mit der in
    # instance_creator gegebenen Methode.
    min_size = 10000
    max_size = 20000
    step = 2000
    instances_per_size = 2

    test_instances = dict()
    for size in range(min_size, max_size + 1, step):
        test_instances[size] = []
        for num in range(instances_per_size):
            array = instance_creator(size)
            test_instances[size].append(array)

    results = dict()

    # Interne Hilfsfunktion, die eine Konfiguraiton auf allen Instanzen laufen
    # lässt, die Zeiten misst und sammelt.
    def collect_data(use_merge_improvement=False, use_insertion_sort=False,
                     use_role_swap=False):
        print("Collecting data for merge sort with parameters",
              "use_merge_improvement =", use_merge_improvement,
              "use_insertion_sort =", use_insertion_sort,
              "use_role_swap =", use_role_swap)

        times = []
        for n in range(min_size, max_size + 1, step):
            cmd = (f"sort_all_test_instances(test_instances[{n}]," +
                   f"use_merge_improvement={use_merge_improvement}," +
                   f"use_insertion_sort={use_insertion_sort},"+
                   f"use_role_swap={use_role_swap})")
            # Wir lassen die Konfiguration dreimal laufen und verwenden die
            # beste Zeit (um die Ergebnisse stabiler zu halten). Da
            # sort_all_test_instances schon instances_per_size viele Durchläufe
            # umfasst, machen wir darüber hinaus nur zehn Wiederholungen in
            # timeit.
            t = timeit.repeat(lambda:
                              sort_all_test_instances(test_instances[n],
                                  use_merge_improvement, use_insertion_sort,
                                  use_role_swap),
                              repeat=3, number=10, globals=globals())
            times.append(min(t)/(10*instances_per_size))
            print("size", n, "took", min(t)/(10*instances_per_size),
                  "seconds on average")
        results[(use_merge_improvement, use_insertion_sort, use_role_swap)] = times


    collect_data(use_merge_improvement=True)
    collect_data(use_insertion_sort=True)
    collect_data(use_role_swap=True)
    collect_data(use_merge_improvement=True, use_insertion_sort=True,
                 use_role_swap=True)
    collect_data()


    xdata =  list(range(min_size, max_size + 1, step))
    plt.plot(xdata, results[(False, False, False)], marker="o", ls="-",
            label="ohne Verbesserung")
    plt.plot(xdata, results[(True, False, False)], marker="o", ls="-",
            label="merge improvement")
    plt.plot(xdata, results[(False, True, False)], marker="o", ls="-",
            label="insertion sort")
    plt.plot(xdata, results[(False, False, True)], marker="o", ls="-",
            label="role swap")
    plt.plot(xdata, results[(True, True, True)], marker="o",
            label="alle Verbesserungen")
    plt.legend(loc="upper left")
    plt.xlabel("Eingabegrösse")
    plt.ylabel("Sekunden")
    plt.show()


Die folgenden Methoden werden zum Estellen eines zufälligen Arrays der gegebenen Grösse mit bestimmten Eigenschaften verwendet.

In [ ]:
# lauter unterschiedliche Einträge in zufälliger Reihenfolge
def random_unique(size):
    array = list(range(size))
    random.shuffle(array)
    return array

# zufällige Einträge aus 10 möglichen Werten
def few_different_values(size):
    return list(random.randrange(10) for i in range(size))

# wir beginnen mit einem sortierten Array, wählen Länge/10 mal
# zwei zufällige Positionen und tauschen ihren Inhalt.
def almost_sorted(size):
    array = list(range(size))
    for iteration in range(size//10):
        i = random.randrange(size)
        j = random.randrange(size)
        array[i], array[j] = array[j], array[i]
    return array


Das erste Experiment führen wir mit den zufälligen Testarrays durch, die unterschiedliche Einträge enthalten (Achtung, das kann eine Weile dauern). Sie können hier erst sinnvolle Messungen durchführen, wenn Sie Aufgaben a-c bearbeitet haben.

In [ ]:
run_experiment(random_unique)

Im nächsten Experiment betrachten wir Arrays mit vielen Duplikaten.

In [ ]:
run_experiment(few_different_values)

Und ein letztes Experiment mit vorsortierten Arrays...

In [ ]:
run_experiment(almost_sorted)